In [1]:
# 𝑎(𝑡) = 𝑅𝑊𝐵(𝑡)( 𝑎_tilde − 𝑏 𝑡 ) + g

In [2]:
import rospy
rospy.init_node("imu_unbiaser")

In [ ]:
import rospy
import tf
from scipy.spatial.transform import Rotation as R
import numpy as np
from std_msgs.msg import Float64MultiArray
from sensor_msgs.msg import Imu
from nav_msgs.msg import Odometry
listener = tf.TransformListener()

g = np.array( [0,0,-9.81] )

odo_m, imu_m, bias_m = [None,], [None,],[None,]
def callback_store(msg, inp):
    inp[0] = msg
rospy.Subscriber("/kimera_vio_ros/odometry", Odometry, callback_store, odo_m )
rospy.Subscriber("/kimera_vio_ros/imu_bias", Float64MultiArray, callback_store, bias_m)
rospy.Subscriber("/versavis/imu", Imu, callback_store, imu_m)

pub = rospy.Publisher("/imu_unbiaser/imu", Imu)

import os

os.system("rosbag play --clock --rate=0.2 -s 5 -u 60 /home/jonfrey/Datasets/labdata/2021-06-17-17-56-33.bag &")
os.system("rosservice call /kimera_vio_ros/kimera_vio_ros_node/restart_kimera_vio &")

r = rospy.Rate(200)
imu_msg = Imu()

count_no_new_imu = 0
while not rospy.is_shutdown():
    try:
        if not odo_m[0] is None and not imu_m[0] is None and not bias_m[0] is None:
            rot = np.array( [ odo_m[0].pose.pose.orientation.x,
                             odo_m[0].pose.pose.orientation.y,
                             odo_m[0].pose.pose.orientation.z,
                             odo_m[0].pose.pose.orientation.w ] )
            # print(rot)
            R_WB = R.from_quat(rot).as_matrix() 

            a = np.array( [ imu_m[0].linear_acceleration.x,
                           imu_m[0].linear_acceleration.y,
                           imu_m[0].linear_acceleration.z] ) 
            b = np.array( bias_m[0].data[3:])

            la = (np.linalg.inv( R_WB ) @ (a-b)) + g             

            if imu_msg.header.seq != imu_m[0].header.seq:
                imu_msg = imu_m[0]
                imu_msg.header.frame_id = "pickelhaubeimu_link"
                imu_msg.linear_acceleration.x = la[0]
                imu_msg.linear_acceleration.y = la[1]
                imu_msg.linear_acceleration.z = la[2]
                pub.publish(imu_msg)
                count_no_new_imu = 0
            else:
                count_no_new_imu += 1
        r.sleep()
        
    except rospy.exceptions.ROSTimeMovedBackwardsException as e:
            print("Ignore time moved backwards! ")
            break
            
    if count_no_new_imu > 500:
                break
print("Done")

<ipython-input-3-be1eb295e1a2>:19: SyntaxWarning: The publisher should be created with an explicit keyword argument 'queue_size'. Please see http://wiki.ros.org/rospy/Overview/Publishers%20and%20Subscribers for more information.
  pub = rospy.Publisher("/imu_unbiaser/imu", Imu)
